In [4]:
!conda env list
!python -V
import sys
print(sys.executable)

# conda environments:
#
base                     /Applications/anaconda3
phd                   *  /Applications/anaconda3/envs/phd

Python 3.10.0
/Applications/anaconda3/envs/phd/bin/python


In [5]:
# LIBRARIES
import numpy as np
import pandas as pd

# OUR OWN LIBRARIES
import utils
import models
from inference_code import beliefPropagation as bp

## Gibbs sampling <a class="anchor" id="3.3"></a>

In [8]:
m = models.lungModel()

# Gibbs inference code
def gibbs_inference_code(I, B, n=100000):

    # Random initialisation
    sample = m.sample()
    inflammations = [sample[I]]
    bacterial_loads = [sample[B]]

    for j in np.arange(1,n):
        # sample from P( bacterial load | (n-1)th inflammation sample )
        point_mass_i = bp.observe(inflammations[j-1], utils.get_second_level_keys(m.cpt_bl_i))
        dist_bl = bp.factor_message_down(point_mass_i, m.cpt_bl_i)
        sample_bl = utils.threeStatesSample(list(dist_bl.values()), list(dist_bl.keys()))
        bacterial_loads.append(sample_bl)

        # use the above bacterial load sample to sample inflammation using P( inflammation | nth bacterial load sample)
        point_mass_bl = bp.observe(bacterial_loads[j], utils.get_first_level_keys(m.cpt_bl_i))
        dist_i = bp.factor_message_up(point_mass_bl, m.cpt_bl_i) # multiply by prior
        # message down from prior
        # compute node belief
        sample_i = utils.threeStatesSample(list(dist_i.values()), list(dist_i.keys()))
        inflammations.append(sample_i)

    return bacterial_loads, inflammations

# Data processing
def get_marginal(name, samples):
    """

    :param name: identifier
    :param samples: a list of values
    :return: distinct values count in list, and related proportions (probability estimate)
    """

    marginal = pd.DataFrame(data={name: samples})
    marginal = pd.DataFrame(data={'count': marginal.value_counts()}).reset_index()
    marginal['P(count)'] = marginal['count']/marginal['count'].sum()
    
    return marginal

In [9]:
# Gibbs sampling implementation for two variables

# Variables
I = 'Inflammation'
B = 'Bacterial load'
    
n_high=1000; n_low=100
b, i = gibbs_inference_code(I, B, n=n_high)
b_fuzzy, i_fuzzy = gibbs_inference_code(I, B, n=n_high)

# Marginal distribution of inflammation
i_marginal = get_marginal(I, i)
i_marginal_fuzzy = get_marginal(I, i_fuzzy)
i_marginal

Observation: {'absent': 1, 'small': 0, 'heavy': 0}
Message down: {'low': 0.6, 'medium': 0.2, 'high': 0.2}
Observation: {'low': 1, 'medium': 0, 'high': 0}
Message up: {'absent': 0.6000000000000001, 'small': 0.30000000000000004, 'heavy': 0.10000000000000002}
Observation: {'absent': 0, 'small': 1, 'heavy': 0}
Message down: {'low': 0.3, 'medium': 0.3, 'high': 0.4}
Observation: {'low': 0, 'medium': 1, 'high': 0}
Message up: {'absent': 0.25, 'small': 0.37499999999999994, 'heavy': 0.37499999999999994}
Observation: {'absent': 1, 'small': 0, 'heavy': 0}
Message down: {'low': 0.6, 'medium': 0.2, 'high': 0.2}
Observation: {'low': 1, 'medium': 0, 'high': 0}
Message up: {'absent': 0.6000000000000001, 'small': 0.30000000000000004, 'heavy': 0.10000000000000002}
Observation: {'absent': 1, 'small': 0, 'heavy': 0}
Message down: {'low': 0.6, 'medium': 0.2, 'high': 0.2}
Observation: {'low': 1, 'medium': 0, 'high': 0}
Message up: {'absent': 0.6000000000000001, 'small': 0.30000000000000004, 'heavy': 0.10000

,Inflammation,count,P(count)
0,absent,344,0.344
1,heavy,329,0.329
2,small,327,0.327


In [10]:
i_marginal_fuzzy

,Inflammation,count,P(count)
0,small,343,0.343
1,heavy,342,0.342
2,absent,315,0.315


In [ ]:
# Marginal distribution of bacterial load
b_marginal = get_marginal(B, b)
b_marginal

In [ ]:
plotFigure(i_marginal, I, column=1, title="Marginal distribution for inflammation for "+str(n_high)+" iterations")
plotFigure(i_marginal_fuzzy, I, column=1, title="Marginal distribution for inflammation "+str(n_low)+" iterations")
plotFigure(b_marginal, B, column=1, title="Marginal distribution for inflammation for "+str(n_high)+" iterations")